# **ARNAV**

## Importing Libraries

In [1]:
import re
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import linear_model
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import re
import string
import nltk
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('omw-1.4',quiet=True)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

!pip --quiet install -U sentence-transformers
from sentence_transformers import SentenceTransformer,CrossEncoder
import pandas as pd

!pip install transformers

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM

!python -m spacy download en_core_web_lg
import spacy

import itertools
import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-02 14:41:25.592722: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Cosine-Similarity Function - cos_sim(emb1,emb2)
### Calculates cos-sim between 2 columns of embeddings

In [2]:
def cos_sim(sentence1_emb, sentence2_emb):
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

## S-BERT CrossEncoder
### sbert_cross(answer, expected_answer)

In [3]:
def sbert_cross(answer, expected_answer):
    # Load the model
    # sbert_cross_model = CrossEncoder('semantic_model')
    sbert_cross_model = CrossEncoder('cross-encoder/stsb-roberta-base')
    similarity = sbert_cross_model.predict([answer,expected_answer])
    return similarity

# **PRATHAM**

## Import the saved model and testing it





For running on our local machine(even CPU), we can just import the saved model and play with it


In [4]:
# obtained_model_roberta=pipeline("text-classification",model="grammer_model")
obtained_model_roberta=pipeline("text-classification",model="imohammad12/GRS-Grammar-Checker-DeBerta")

## Define RoBERTa function

In [5]:
def roberta(sentence):  
    o=obtained_model_roberta(sentence)
    ans=o[0]["score"]
    return ans

# **HAARISH**

In [6]:
nlp = spacy.load("en_core_web_lg")

## Extract POS

In [7]:
def extract_POS(sample_doc):
    res=[]
    for chk in sample_doc.noun_chunks:
        tmp=""
        for tkn in chk:
            if (tkn.pos_ in ['NOUN','PROPN','ADJ'] ):
                if (not(tkn.is_stop) and not(tkn.is_punct)):
                    tmp = tmp + tkn.text.lower() + " "
        if(tmp.strip()!=""):
            res.append(tmp.strip())
    return list(dict.fromkeys(res))

## Match Keywords

In [8]:
def matching_keywords(stdlst,keylst):
        #matched list
        res=[]
        for x in stdlst:
            # res += list(set([key for key in keylst if re.search(x, key)]))
            for key in keylst:
                if(re.search(re.escape(x),re.escape(key)) or re.search(re.escape(key),re.escape(x))):
                    res.append(key)
        return res

## Assign weights to keywords

In [9]:
def dictionary_with_weights(words):
  
  # nouns are given weightage as 9
  # proper nouns are given weightage as 8.5
  # adjectives are given weightage as 5

  categorized_words = {'9': [], '8.5': [], '5': []}
  for word in words:
      doc = nlp(word)
      pos = doc[0].pos_
      if pos in ['NOUN', 'PRON']:
          categorized_words['9'].append(word)
      elif pos == 'PROPN':
          categorized_words['8.5'].append(word)
      elif pos == 'ADJ':
          categorized_words['5'].append(word)
  return categorized_words


## Score the answer

In [10]:
def keyword_scoring(stud_ans,ans_key):

  ans_doc=nlp(ans_key)
  key_POS=extract_POS(ans_doc)
  

  model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  doc_embedding = model.encode([ans_key])
  candidate_embeddings = model.encode(key_POS)
  top_n = 7
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [key_POS[index] for index in distances.argsort()[0][-top_n:]]
  keywords_scores = dictionary_with_weights(keywords)
  

  total=0
  count=0

  for i in keywords_scores['9']:
    count+=1
  total=total+count*9

  count=0
  for i in keywords_scores['8.5']:
    count+=1
  total=total+count*8.5

  count=0
  for i in keywords_scores['5']:
    count+=1
  total=total+count*5

  
  if(type(stud_ans)==float):
    stud_ans=str(stud_ans)

  print(keywords_scores)
  stud_doc=nlp(stud_ans) 
  std_POS=extract_POS(stud_doc)
  #then apply match function
  matched=matching_keywords(std_POS,keywords)
  stud_score=0
  #percentage of matched keywords along with weights
  for i in matched:
    if i in keywords_scores['9']:
      stud_score=stud_score+9
    elif i in keywords_scores['8.5']:
      stud_score=stud_score+8.5
    elif i in keywords_scores['5']:
      stud_score=stud_score+5 
  score=stud_score/total
  return score

# **Using Regression Model from saved file**

In [11]:
#change the key for regressor and 'out_of' to get diff o/p
#any permuation and combination will work
def finalMarks1(li):
    out_of = 7
    regressor={"lr7":"LinReg7.pickle","rf7":"RandomForest7.pickle","xgb7":"XGBoost7.pickle",
           "lr5":"LinReg5.pickle","rf5":"RandomForest5.pickle","xgb5":"XGBoost5.pickle",
           "lr3":"LinReg3.pickle","rf3":"RandomForest3.pickle","xgb3":"XGBoost7.pickle"}

    loaded_model = pickle.load(open(regressor['rf'+str(out_of)],'rb')) 
    x=loaded_model.predict(li)
    np.round(x)
    x=x.clip(0,out_of)
    x=int(x)
    print(x,f"out of {out_of}")
    return x

In [13]:
#this version takes the marks out of 7 and converts it to out of 3
#just change the key for regressor to get diff o/p
def finalMarks2(li):
    regressor={"lr7":"LinReg7.pickle","rf7":"RandomForest7.pickle","xgb7":"XGBoost7.pickle"}
    
    loaded_model = pickle.load(open(regressor['rf7'],'rb')) 
    x=loaded_model.predict(li)
    np.round(x)
    x=x.clip(0,7)
    x=(x/7)*3
    x=int(x)
    print(x,f"out of 3")
    return x

In [ ]:
#should work best according to statistics
def finalMarks3(li):
    regressor={"lr7":"LinReg7.pickle","rf7":"RandomForest7.pickle","xgb7":"XGBoost7.pickle",
           "lr5":"LinReg5.pickle","rf5":"RandomForest5.pickle","xgb5":"XGBoost5.pickle",
           "lr3":"LinReg3.pickle","rf3":"RandomForest3.pickle","xgb3":"XGBoost7.pickle"}

    rf3 = pickle.load(open(regressor['rf3'],'rb')) 
    xgb7to3 = pickle.load(open(regressor['xgb7'],'rb')) 

    rf3_x=rf3.predict(li)
    np.round(rf3_x)
    rf3_x=rf3_x.clip(0,3)
    rf3_x=int(rf3_x)

    xgb7to3_x=xgb7to3.predict(li)
    xgb7to3_x=(xgb7to3_x/7)*3
    np.ceil(xgb7to3_x)
    xgb7to3_x=xgb7to3_x.clip(0,3)
    xgb7to3_x=int(xgb7to3_x)

    x=max(xgb7to3_x,rf3_x)
    print(x,f"out of 3")
    return x

# **Final Marks**

In [14]:
def descAnswerEval(ans,key):
    return finalMarks1([sbert_cross(ans,key),roberta(ans),keyword_scoring(ans,key)])
